Once you are satisfied with a Decision Optimization scenario that solves successfully with expected results, you can now deploy it for use by an application inside or outside of DSX Local.
To deploy, you first need to publish the scenario as a deployable model. This can be done in the UI, or by code.

In [1]:
print('publish scenario as deployable model (can be done with UI)')

print('retrieves our scenario')
from dd_scenario import Client as ScenarioClient
sclient = ScenarioClient()
do_model = sclient.get_decision_frameworks(name='Diet')[0]
scenario = do_model.get_scenario(name='Scenario 1')

print('creates a deployer client')
from dd_deployer import Client as DeployerClient
dclient = DeployerClient()
for p in dclient.get_deployable_models(do_model, scenario):
    print('deleting previous published model %s' % p.name)
    dclient.delete_deployable_model(p)           
print('publishing Scenario 1 as diet_scenario1_published')
published_model = dclient.save_deployable_model(do_model, scenario, name='diet_scenario1_published')

publish scenario as deployable model (can be done with UI)
retrieves our scenario
creates a deployer client
deleting previous published model diet_scenario1_published
publishing Scenario 1 as diet_scenario1_published


Once the scenario is published, you are ready to deploy.
The deployment has two steps:
- create a runtime: this will start one (or several) optimization job processor(s) on the cluster with a reserved a capacity of memory and CPUs. Those will be used to solve the deployed models. You can use a runtime for several models, or create a new runtime for each model. When solving the models, the solves are put in a queue, and solved as soon as a job processor is available, some more capacity means less wait time.
- deploy the published model for use on the given runtime. Once deployed, the optimization model is ready to be used and solved several times with different data.

In [2]:
print('Retrieve published model')
published_model=dclient.get_deployable_model(name='diet_scenario1_published')
print('published_model is %s' % published_model)

print('clean any previous runtimes:')
for r in dclient.get_runtime_environments():
    print('deleting runtime environement %s' % r.name)
    dclient.delete_runtime_environment(r)
print('Create runtime environment: we will use one processor with 4 cores and 4 Gb of memory, able to run one job in parallel')
runtime_env = dclient.create_runtime_environment(name='diet_runtime',number_of_processors=1,workers_per_processor=1,cpus_per_processor=4,memory_per_processor=4)
print('runtime_env is %s' % runtime_env)

print('clean any previous deployments:')
for d in dclient.get_deployable_model_deployments():
    print('deleting deployment %s' % d.deployableModelId)
    dclient.delete_deployable_model_deployment(d)
print('Create model deployment')
model_deployment = dclient.create_deployable_model_deployment(name='diet_deployed',runtime_environment=runtime_env,deployable_model=published_model)
print('deployment is %s' % model_deployment)

executionServiceURL=model_deployment.executionServiceURL
executionServiceApiKey=model_deployment.executionServiceDeployedModelApiKey
print('You can solve on %s with key %s' % (executionServiceURL,executionServiceApiKey))
print('(if you are calling from outside of the cluster, replace "ibm-nginx-svc.ibm-private-cloud.svc.cluster.local.svc.cluster.local" in the URL by the cluster portal IP)')

Retrieve published model
published_model is {
   "inputSchemas": [
      {
         "fields": [
            {
               "type": "string", 
               "name": "name", 
               "nullable": true
            }, 
            {
               "type": "double", 
               "name": "qmin", 
               "nullable": true
            }, 
            {
               "type": "double", 
               "name": "qmax", 
               "nullable": true
            }
         ], 
         "type": "struct", 
         "name": "diet_nutrients"
      }, 
      {
         "fields": [
            {
               "type": "string", 
               "name": "Food", 
               "nullable": true
            }, 
            {
               "type": "double", 
               "name": "Calories", 
               "nullable": true
            }, 
            {
               "type": "double", 
               "name": "Calcium", 
               "nullable": true
            }, 
            {
   

Now that the model is deployed, it is ready for use in your application.
The model can be called from inside or outside DSX, with the URL and API Returned by the deployment.
A simple REST API is provided to submit input data, solve and get results.
A client python library is provided inside DSX to further ease the process.
Its use is show below.

In [3]:
# Let's use now the deployed service
from dd_execution import Client as ExecutionClient
execution_client = ExecutionClient(api_url=executionServiceURL,
                                   api_key=executionServiceApiKey)
# solve config
# use default solve config for the client, which will be
# the default solve config found at executionServiceURL
deployment_desc = execution_client.get_deployment_description()

# for the sake of printing some info
solve_config = deployment_desc['deploymentDescription']['solveConfig']
print('Solve config: %s' % solve_config)

# solve
solve_data = {}
import pandas
diet_food_columns=["name","unit_cost","qmin","qmax"]
diet_food_data=[
    ("Roasted Chicken", 0.84, 0, 10),
    ("Spaghetti W/ Sauce", 0.78, 0, 10),
    ("Tomato,Red,Ripe,Raw", 0.27, 0, 10),
    ("Apple,Raw,W/Skin", .24, 0, 10),
    ("Grapes", 0.32, 0, 10),
    ("Chocolate Chip Cookies", 0.03, 0, 10),
    ("Lowfat Milk", 0.23, 0, 10),
    ("Raisin Brn", 0.34, 0, 10),
    ("Hotdog", 0.31, 0, 10)]
solve_data['diet_food'] = pandas.DataFrame.from_records(diet_food_data,columns=diet_food_columns)

diet_nutrients_columns=["name","qmin","qmax"]
diet_nutrients_data=[
    ("Calories", 2000, 2500),
    ("Calcium", 800, 1600),
    ("Iron", 10, 30),
    ("Vit_A", 5000, 50000),
    ("Dietary_Fiber", 25, 100),
    ("Carbohydrates", 0, 300),
    ("Protein", 50, 100)]
solve_data['diet_nutrients'] = pandas.DataFrame.from_records(diet_nutrients_data,columns=diet_nutrients_columns)

diet_food_nutrients_columns=["Food","Calories","Calcium","Iron","Vit_A","Dietary_Fiber","Carbohydrates","Protein"]
diet_food_nutrients_data=[
    ("Roasted Chicken", 277.4, 21.9, 1.8, 77.4, 0, 0, 42.2),
    ("Spaghetti W/ Sauce", 358.2, 80.2, 2.3, 3055.2, 11.6, 58.3, 8.2),
    ("Tomato,Red,Ripe,Raw", 25.8, 6.2, 0.6, 766.3, 1.4, 5.7, 1),
    ("Apple,Raw,W/Skin", 81.4, 9.7, 0.2, 73.1, 3.7, 21, 0.3),
    ("Grapes", 15.1, 3.4, 0.1, 24, 0.2, 4.1, 0.2),
    ("Chocolate Chip Cookies", 78.1, 6.2, 0.4, 101.8, 0, 9.3, 0.9),
    ("Lowfat Milk", 121.2, 296.7, 0.1, 500.2, 0, 11.7, 8.1),
    ("Raisin Brn", 115.1, 12.9, 16.8, 1250.2, 4, 27.9, 4),
    ("Hotdog", 242.1, 23.5, 2.3, 0, 0, 18, 10.4)]
solve_data['diet_food_nutrients'] = pandas.DataFrame.from_records(diet_food_nutrients_data,columns=diet_food_nutrients_columns)


print('solve_data = %s' % solve_data)

job = execution_client.solve(solve_config=solve_config,
                             input_data=solve_data)
print('JOB = %s' % job)
job.wait_for_completion()
# check status
if job.info['solveState']['executionStatus'] != 'PROCESSED':
    raise ValueError('Bad solve status, failureinfo = %s' % job.info['solveState']['failureInfo'])
# check job attachments
solution_df = job.output['solution']
print('SOLUTION: %s' % solution_df)

input = https://ibm-nginx-svc.ibm-private-cloud.svc.cluster.local/v1/dopt-execution-api/model/8ac1e35d-06ab-4fa5-bc9f-2e1c4510b584
using url = https://ibm-nginx-svc.ibm-private-cloud.svc.cluster.local/v1/dopt-execution-api
Solve config: {u'workerType': u'docplex', u'attachments': [{u'category': u'input', u'type': u'CONTAINER', u'name': u'models', u'containerId': u'8ac1e35d-06ab-4fa5-bc9f-2e1c4510b584-models'}]}
solve_data = {'diet_food':                      name  unit_cost  qmin  qmax
0         Roasted Chicken       0.84     0    10
1      Spaghetti W/ Sauce       0.78     0    10
2     Tomato,Red,Ripe,Raw       0.27     0    10
3        Apple,Raw,W/Skin       0.24     0    10
4                  Grapes       0.32     0    10
5  Chocolate Chip Cookies       0.03     0    10
6             Lowfat Milk       0.23     0    10
7              Raisin Brn       0.34     0    10
8                  Hotdog       0.31     0    10, 'diet_nutrients':             name  qmin   qmax
0       Calories  2

Finally we remove our runtime and deployment from the cluster, to save the resources for other users

In [4]:
print('clean any running runtimes:')
for r in dclient.get_runtime_environments():
    print('deleting runtime environement %s' % r.name)
    dclient.delete_runtime_environment(r)

print('clean any deployments:')
for d in dclient.get_deployable_model_deployments():
    print('deleting deployment %s' % d.deployableModelId)
    dclient.delete_deployable_model_deployment(d)

clean any running runtimes:
deleting runtime environement diet_runtime
clean any deployments:
deleting deployment 5f49512e-4c1f-447e-9295-2f248dee251b
